In [1]:
import requests

In [4]:
host = 'localhost'

In [5]:
assert requests.get('http://%s:4324/health'%host).status_code == 200

In [6]:
resp = requests.post('http://%s:4324/verify'%host, '{"signedCredentials":{"issuer":"a"}}')
print(resp.status_code)
assert resp.status_code == 400 
assert resp.content

400


In [9]:
import json
creds = json.loads('{"@context":["https://www.w3.org/2018/credentials/v1",{"name":"schema:name"}],"type":["VerifiableCredential"],"credentialSubject":{"type":"Person","name":"Tejash"},"issuanceDate":"2021-08-27T10:57:57.237Z","issuer":"did:authorizedIssuer:23423#21","proof":{"type":"Ed25519Signature2018","created":"2022-07-19T05:11:21Z","verificationMethod":"did:authorizedIssuer:23423#21","proofPurpose":"assertionMethod","jws":"eyJhbGciOiJFZERTQSIsImI2NCI6ZmFsc2UsImNyaXQiOlsiYjY0Il19..O6u5oAOiOeOt4nCHlJRRn04EyOoi7sRZHcJtvRXQqvKxQUfKdYjB9GJunO5pjnErWJC4U7gTctmUq34qrx98Bw"}}')
resp = requests.post('http://%s:4324/verify'%host, json={"signedCredentials":creds})
assert resp.status_code==200
assert resp.json()["verified"]
resp.content
print("PASS")

PASS


In [15]:
resp = requests.post('http://%s:4324/sign'%host, json={"signedCredentials":""})
assert resp.status_code == 400

In [40]:
template="""{
        "@context": [
            "https://www.w3.org/2018/credentials/v1",
            {
                "@context": {
                    "@version": 1.1,
                    "@protected": true,
                    "SkillCertificate": {
                        "@id": "https://github.com/sunbird-specs/vc-specs#SkillCertificate",
                        "@context": {
                            "id": "@id",
                            "@version": 1.1,
                            "@protected": true,
                            "skills": {
                                "@id": "https://github.com/sunbird-specs/vc-specs#skills",
                                "@container": "@list"
                            },
                            "name":"schema:Text",
                            "refId":"schema:Text"
                        }
                    },
                    
                    "trainedOn": {
                        "@id": "https://github.com/sunbird-specs/vc-specs#trainedOn",
                        "@context": {
                            "name": "schema:Text"
                        }
                    }
                }
            }
        ],
        "type": [
            "VerifiableCredential","SkillCertificate"
        ],
        "credentialSubject": {
            "type": "SkillCertificate",
            "refId": "{{id}}",
            "name": "{{name}}",
            "skills": [{{#each skills}}"{{skill}}"{{#unless @last}},{{/unless}}{{/each}}]
        },
        "issuanceDate": "{{date}}",
        "issuer": "did:authorizedIssuer:23423#21",
        "evidence": [
            {
                "type": [
                    "Vaccination"
                ],
                "id": "https://sunbird.org/id"
            }
        ]
    }"""

In [41]:
entity = {
        "name": "a",
        "id": "123",
        "skills": [{"skill":"a"},{"skill":"b"}],
        "date":"2022-07-01T12:12:00Z"
    }

In [43]:
resp = requests.post('http://%s:4324/sign'%host, json={"data":entity, "credentialTemplate":template})
print(resp.content)
assert resp.status_code == 200
resp.json()['proof']['type'] == 'Ed25519Signature2018'

b'{"@context":["https://www.w3.org/2018/credentials/v1",{"@context":{"@version":1.1,"@protected":true,"SkillCertificate":{"@id":"https://github.com/sunbird-specs/vc-specs#SkillCertificate","@context":{"id":"@id","@version":1.1,"@protected":true,"skills":{"@id":"https://github.com/sunbird-specs/vc-specs#skills","@container":"@list"},"name":"schema:Text","refId":"schema:Text"}},"trainedOn":{"@id":"https://github.com/sunbird-specs/vc-specs#trainedOn","@context":{"name":"schema:Text"}}}}],"type":["VerifiableCredential","SkillCertificate"],"credentialSubject":{"type":"SkillCertificate","refId":"123","name":"a","skills":["a","b"]},"issuanceDate":"2022-07-01T12:12:00Z","issuer":"did:authorizedIssuer:23423#21","evidence":[{"type":["Vaccination"],"id":"https://sunbird.org/id"}],"proof":{"type":"Ed25519Signature2018","created":"2022-07-29T06:59:51Z","verificationMethod":"did:authorizedIssuer:23423#21","proofPurpose":"assertionMethod","jws":"eyJhbGciOiJFZERTQSIsImI2NCI6ZmFsc2UsImNyaXQiOlsiYjY0Il1

True

In [35]:
resp.json()

{'@context': ['https://www.w3.org/2018/credentials/v1',
  {'@context': {'@version': 1.1,
    '@protected': True,
    'SkillCertificate': {'@id': 'https://github.com/sunbird-specs/vc-specs#SkillCertificate',
     '@context': {'id': '@id',
      '@version': 1.1,
      '@protected': True,
      'skills': {'@id': 'https://github.com/sunbird-specs/vc-specs#skills',
       '@container': '@list'},
      'name': 'schema:Text',
      'refId': 'schema:Text'}},
    'trainedOn': {'@id': 'https://github.com/sunbird-specs/vc-specs#trainedOn',
     '@context': {'name': 'schema:Text'}}}}],
 'type': ['VerifiableCredential', 'SkillCertificate'],
 'credentialSubject': {'type': 'SkillCertificate',
  'refId': '123',
  'name': 'a',
  'skills': ['a', 'b']},
 'issuanceDate': '2021-08-27T10:57:57.237Z',
 'issuer': 'did:authorizedIssuer:2342233#21',
 'evidence': [{'type': ['Vaccination'], 'id': 'https://sunbird.org/id'}],
 'proof': {'type': 'RsaSignature2018',
  'created': '2022-07-29T06:57:43Z',
  'verificatio

In [45]:
assert resp.json()["@context"][0] == 'https://www.w3.org/2018/credentials/v1'